In [1]:
import pymysql

class Student:
    def __init__(self):
        self.bunho = 0
        self.name = ''
        self.kor = 0
        self.mat = 0
        self.eng = 0
        self.total = 0
        self.average = 0
        self.grade = ''
        
    def inputData(self):
        self.name = input("이름을 입력하시오 : ")
        self.kor = eval(input("국어점수 : "))
        self.mat = eval(input("수학점수 : "))
        self.eng = eval(input("영어점수 : "))
        
    def calc_total_average(self):
        self.total = self.kor + self.mat + self.eng
        self.average = round(self.totla / 3, 2)
        self.grade = self.calcSemGrade()
        
    def calcSemGrade(self):
        if self.average >= 90:
            return "A"
        elif self.average >= 80:
            return "B"
        elif self.average >= 70:
            return "C"
        elif self.average >= 60:
            return "D"
        else :
            return "F"
          
    def __str__(self):      # % 뒤에 데이터 불러오기. 5: 5자리 수?
        return '%5s %5s %6s %6s %6s %7s %7s %s' % (self.bunho, self.name, self.kor, 
                                               self.mat, self.eng, self.total, self.average, self.grade)
    
    def __cmp__(self, other):    # 새로운 데이터 입력하고 계속 넣을때 사용
        return self.name == other.name
    
    def __call__(self, other):     # class를 함수처럼 사용하고자 할때 오버라이딩
        st = []     # empty list
        for student in other:
            stp = Student()
            stp.name = student[0]
            stp.kor = student[1]
            stp.mat = student[2]
            stp.eng = student[3]
            st.append(stp)
        return st

# sungjuk 테이블

In [2]:
class maria:
    schoolname = "에이콘 아카데미"
    def __init__(self):
        self.conn = pymysql.connect(host='192.168.1.243', port=3306, user='root',
                           password='big2019',db='acornschool', charset='utf8',
                                    autocommit=True)
        cor = self.conn.cursor()   
        cor.execute("SHOW TABLES")
        tables = cor.fetchall()
        c = []
        for i in tables:
            if  i == ('sungjuk',):
                c.append(i)
        if len(c) != 0:
            sungjuk_table = "SELECT * FROM sungjuk"
            cor.execute(sungjuk_table)
            print('테이블이 있습니다. ')
        else:
            cor.execute("CREATE TABLE sungjuk (bunho INTEGER(11) AUTO_INCREMENT,name VARCHAR(20), kor INTEGER(20),  mat INTEGER(20),eng INTEGER(20))")
            print('sungjuk테이블을 생성하였습니다. ')
    def __del__(self):
        self.conn.close()

    def selectdb(self ):
        cur = self.conn.cursor()
        cur.callproc("sungjuk_select")
        if (cur.rowcount):
            stu = list(cur.fetchall())
        else :
            stu = 0;
        cur.close()
        return stu
   
    def insertdb(self,name, kor, mat, eng):
        cur = self.conn.cursor()
        args = (name, kor, mat, eng)
        cur.callproc("sungjuk_insert",args)
        cur.close()
       
    def updatedb(self, in_name, up_name):
        cur = self.conn.cursor()
        args = (in_name, up_name, 0)
        cur.callproc("sungjuk_update", args)
        cur.execute('SELECT @_sungjuk_update_2')
        result = cur.fetchone()
        if result==2 :
            print("수정할 이름이 없습니다.")
        else :
            print("성공적으로 수정하였습니다. ")
        cur.close()
        return result 
   
    def deletedb(self,in_name):
        cur = self.conn.cursor()
        args = (in_name, 0)
        cur.callproc("sungjuk_delete", args)
        cur.execute('SELECT @_sungjuk_delete_1')
        result = cur.fetchone()
        if result==2 :
            print("삭제할 이름이 없습니다.")
        else :
            print("성공적으로 삭제하였습니다. ")
        cur.close()
        return result
             

# management & class 만들어서 데이터 관리

In [3]:
from math import  *
from statistics import *

class Management:
    def __init__(self):
        self.maria = maria()
        
    def M_select(self):
        data = self.maria.selectdb()
        st = []
        if (data):
            for student in data:
                stp = Student()
                stp.bunho = student[0]
                stp.name = student[1]
                stp.kor = student[2]
                stp.eng = student[3]
                stp.mat = student[4]
                stp.total = student[5]
                stp.average = student[6]
                stp.grade = student[7]
                st.append(stp)                    
            # studdata=Student((data))
            print(maria.schoolname + "  성적 계산표 ")
            print('%5s %5s %6s %6s %6s %6s  %6s %6s' % ('번 호', '이 름', '국 어',
                    '수 학', '영  어', '총 점', '평 균', '학 점' ))      
            for stud in st:
                print( stud)                     
        else :
            print("데이터가 없습니다. . ")
           
    def M_insert(self):    # student 호출되서 insert
        stu = Student()
        stu.inputData()
        self.maria.insertdb(stu.name, stu.kor, stu.mat, stu.eng)
        
    def M_calculate(self):
        stu = Student()
        stu.calc_total_average()
        self.maria.insertdb(stu.total, stu.average)

        
    def M_update(self):
        name = input("검색할 이름을 입력하시요")
        replace_name = input("수정할 이름을 입력하시요")
        result=self.maria.updatedb(name, replace_name)    # updatedb하면 db 에 저장된다
        
    def M_delete(self):       
        name = input("검색할 이름을 입력하시요")
        self.maria.deletedb(name)     



# 메뉴만들기

In [4]:
import sys
class Menu():
    def __init__(self):
        self.manage = Management()
       
    def work(self):
        menu = [ "전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)"]
        manage = Management()
        while 1:
            sel = input(menu)
            if  sel == "1":
                manage.M_select()
                continue
            elif sel == "2":
                manage.M_insert()
                continue       
            elif sel =="3":
                manage.M_update()
                continue
            elif sel =="4":
                manage.M_delete()
            elif sel =="9":
                del(manage)
                sys.exit()
            else :
                print("잘못된 입력입니다. 다시입력 하시요")
                continue
           
menu = Menu()
menu.work()

테이블이 있습니다. 
테이블이 있습니다. 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']2
이름을 입력하시오 : 하니
국어점수 : 12
수학점수 : 12
영어점수 : 34
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']1
에이콘 아카데미  성적 계산표 
  번 호   이 름    국 어    수 학   영  어    총 점     평 균    학 점
   14    아리    100    100    100    None    None None
   15   장주혜    100    100    100    None    None None
   16    하나     30     50      1    None    None None
   17    하니     12     34     12    None    None None
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']4
검색할 이름을 입력하시요하니
성공적으로 삭제하였습니다. 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']1
에이콘 아카데미  성적 계산표 
  번 호   이 름    국 어    수 학   영  어    총 점     평 균    학 점
   14    아리    100    100    100    None    None None
   15   장주혜    100    100    100    None    None None
   16    하나     30     50      1    None    None None
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']9


SystemExit: 

C:\Users\ICT01_09\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
